# Segmenting and Clustering Mining Towns in Canada


Project Description:

Minerals and metals are the building blocks of our modern society and the mining industry is one of the highest-paid industries within Canada. However since mining jobs are only found where there is mining, mining professionals usually have to move to a mining town/city, which is usually located in remote regions. Despite the huge progress made to grant miners fair living conditions, mining towns or mining cities are not always fun.
    
In this project, I will **explore and categorize the condition of the mining towns in Canada.** Each town will be categorized based on its amenities. This will help mining professionals to make an informed choice on their work locations.
    
Data Source:
Part 1 : I will use the ArcGIS REST API to generate a dataframe of the mine as my inital starting point.
- Current Operating Mine and their location: ArcGIS REST API
     - This layer is maintained by the Government of Canada 
     - https://maps-cartes.services.geo.ca/server_serveur/rest/services/NRCan/900A_and_top_100_en/MapServer
     - https://open.canada.ca/data/en/dataset/000183ed-8864-42f0-ae43-c4313a860720#wb-auto-6
     
Part 2: I will use the geocode package to find the address of the mine and their associated town .

Part 3: I will use the town address to call the Foursquare API to extract the amenities data.

Part 4: I will use kmean and folium to group and visulaize the results.
- Town Amenities Data : Foursquare API
- Visualization: Folium

In [1]:
#Data processing libraries
import pandas as pd
import numpy as np

#Geo Data processing libraries
from pyproj import Transformer #Converting Spatial Reference 
from geopy.geocoders import Nominatim #Geo location search
import geocoder # import geocoder

# library to handle requests
import requests 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# map rendering library
import folium 

In [2]:
### Using ArcGIS REST APIs to obtain all the producing mines and it's geo location
dict_MineType = {'Metal mines':4,'Nonmetal mines':5,'Coal mines':6,'Oil sands mines':7}
mine_list=[]

for key,value in dict_MineType.items():
    url = ("https://maps-cartes.services.geo.ca/server_serveur/rest/services/NRCan/900A_and_top_100_en/MapServer/{}/query?where=&text=%25&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&having=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=pjson".format(value))
    results = requests.get(url).json()
    for j in results['features']:
        mine_list.append([j['attributes']['operation_name_En'],j['geometry']['x'],j['geometry']['y'],key,value])


In [3]:
df_Mine = pd.DataFrame(mine_list,columns=['Mine Name','x','y','Mine Type','Mine Type ID'])

In [4]:
#Converting projected coordinates to lat/lon using pyproj
transformer = Transformer.from_crs('epsg:3978', 'epsg:4326')
x2,y2 = transformer.transform(df_Mine['x'].tolist(), df_Mine['y'].tolist())
df_Mine['Latitude'] = x2
df_Mine['Longitude'] = y2

In [5]:
df_Mine.tail(15)

,Mine Name,x,y,Mine Type,Mine Type ID,Latitude,Longitude
194,Wolverine,-1.597623e+06,1.004570e+06,Coal mines,6,55.086100,-121.245300
195,Brule,-1.617278e+06,1.050522e+06,Coal mines,6,55.398000,-121.832000
196,Willow Creek,-1.629892e+06,1.081255e+06,Coal mines,6,55.607000,-122.222200
197,Donkin,2.610312e+06,4.251636e+05,Coal mines,6,46.178116,-59.825858
198,Stellarton,2.451324e+06,2.519963e+05,Coal mines,6,45.566075,-62.682864
199,Estevan,-5.728299e+05,4.517093e+04,Coal mines,6,49.087329,-102.864957
200,Poplar River,-7.588761e+05,8.680077e+04,Coal mines,6,49.217566,-105.472584
201,Kearl,-9.327636e+05,1.040357e+06,Oil sands mines,7,57.396908,-111.066731
202,Millennium and Steepbank,-9.627859e+05,9.891823e+05,Oil sands mines,7,56.871900,-111.340001
203,Jackpine,-9.546364e+05,1.029449e+06,Oil sands mines,7,57.249036,-111.379858


In [6]:
df_Mine['marker_color'] = pd.cut(df_Mine['Mine Type ID'], bins=4, labels=['yellow', 'green', 'blue', 'red'])

In [7]:
# create map
map_mine_in_canada = folium.Map(location=[57, -91], zoom_start=4)

# add markers to the map
for lat, lon, poi, mcolor,mtype in zip(df_Mine['Latitude'], df_Mine['Longitude'],df_Mine['Mine Name'], df_Mine['marker_color'],df_Mine['Mine Type']):
    label = folium.Popup(str(poi) + ' ' + str(mtype))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=mcolor,
        fill=True,
        fill_color=mcolor,
        fill_opacity=0.7).add_to(map_mine_in_canada)
    
map_mine_in_canada

In [8]:
#https://developers.arcgis.com/rest/geocode/api-reference/geocoding-reverse-geocode.htm
#https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/reverseGeocode?f=pjson&featureTypes=&location=-111.340000,56.871900
l_results = []
for mine_name,lat, lonin in zip(df_Mine['Mine Name'],df_Mine['Latitude'], df_Mine['Longitude']):
    arcgis_url = ("https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/reverseGeocode?f=pjson&featureTypes=&location={},{}".format(lonin,lat))
    results = requests.get(arcgis_url).json()
    l_results.append([mine_name,results['address']['LongLabel'],results['address']['City'],results['address']['MetroArea'],results['address']['Subregion'],results['address']['Region'],results['address']['Territory']])

In [9]:
df_Mine_Town = pd.DataFrame(l_results,columns=['Mine Name','LongLabel','City','MetroArea','Subregion','Region','Territory'])

In [10]:
df_Mine['Town address'] = df_Mine_Town['City'] +' , ' +df_Mine_Town['Region']

In [14]:
l_town_lat = []
l_town_long = []
for town_address in zip(df_Mine['Town address']):
    address_converter = Nominatim(user_agent = 'foursquare_agent')
    location = address_converter.geocode(town_address)
    latitude = location.latitude
    longitude = location.longitude
    l_town_lat.append(latitude)
    l_town_long.append(longitude)

In [16]:
df_Town_Coord = pd.DataFrame({'lat':l_town_lat,'long':l_town_long})
df_Mine['Town lat'] = df_Town_Coord['lat']
df_Mine['Town long'] = df_Town_Coord['long']

In [17]:
df_Mine.head()

,Mine Name,x,y,Mine Type,Mine Type ID,Latitude,Longitude,marker_color,Town address,Town lat,Town long
0,New Afton,-1.745331e+06,5.389900e+05,Metal mines,4,50.661000,-120.514000,yellow,"Kamloops , British Columbia",50.675827,-120.339415
1,Copper Mountain,-1.804148e+06,4.038439e+05,Metal mines,4,49.331001,-120.533002,yellow,"Fraser Valley , British Columbia",59.715297,-135.047995
2,Highland Valley,-1.787566e+06,5.359916e+05,Metal mines,4,50.485500,-121.048300,yellow,"Logan Lake , British Columbia",50.494463,-120.813366
3,Gibraltar,-1.773727e+06,7.764453e+05,Metal mines,4,52.528495,-122.287160,yellow,"Cariboo , British Columbia",49.247779,-122.889774
4,Mount Milligan,-1.754343e+06,1.081143e+06,Metal mines,4,55.118000,-124.031000,yellow,"Bulkley-Nechako , British Columbia",54.531617,-125.605626


### Part 3: Segmenting the Clustering Neighborhoods in Downtown Toronto 
- Data Source: **four square API**
- Algorithm: **Kmeans**
- Visualization: **Folium**

In [26]:
import config #Four square API key
VERSION = '20180604'
LIMIT = 100


In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=50000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            config.FSQ_CLIENT_ID, 
            config.FSQ_CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
Mine_venues = getNearbyVenues(names=df_Mine['Mine Name'],
                                   latitudes=df_Mine['Town lat'],
                                   longitudes=df_Mine['Town long']
                                  )

New Afton
Copper Mountain
Highland Valley
Gibraltar
Mount Milligan 
Myra Falls
Red Chris
Brucejack
Silvertip
Tanco
Thompson (T-1 and T-3)
Lalor Lake
777
Caribou
Beaver Brook
Nugget Pond
Ming
Point Rousse (Pine Cove)
Voisey’s Bay
Scully
Carol Lake
DSO (Timmins)
Moose River Consolidated
Mary River
Meliadine
Meadowbank
Amaruq
Hope Bay
Detour Lake
Holloway-Holt
Macassa
Black Fox
Taylor
Young-Davidson
Nickel Rim South
Garson
Clarabelle
Copper Cliff North
Hoyle Pond
Bell Creek
Creighton
Hollinger
Coleman
Fraser
Strathcona
Kidd Creek
McCreedy West
Totten
Timmins West
Borden
Island
Sugar Zone
Mishi
Eagle River
Hemlo (Williams)
Lac des Iles
Musselwhite
Red Lake
Cochenour
Rainy River
Tio
Bloom Lake
Mont-Wright
DSO (Goodwood)
Fire Lake
Niobec
Raglan
Nunavik
Éléonore
Langlois
Bracemac-McLeod
Lamaque
Goldex
Sleeping Giant
Canadian Malartic
LaRonde
LaRonde - Zone 5
Westwood-Doyon
Elder
Casa Berardi
Santoy
Seabee
McClean Lake
Cigar Lake
Keno Hill Silver District
Eagle (Dublin Gulch)
Minto
Lindbergh
S

In [32]:
Mine_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New Afton,50.675827,-120.339415,The Noble Pig Brewhouse & Restaurant,50.675879,-120.325953,Brewery
1,New Afton,50.675827,-120.339415,Tournament Capital Centre (TCC),50.668740,-120.365730,Gym
2,New Afton,50.675827,-120.339415,The Chopped Leaf,50.664296,-120.355503,Salad Place
3,New Afton,50.675827,-120.339415,Earls Restaurant,50.665026,-120.353525,American Restaurant
4,New Afton,50.675827,-120.339415,Brownstone Restaurant,50.676427,-120.338737,French Restaurant


In [33]:
# one hot encoding
Mine_onehot = pd.get_dummies(Mine_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Mine_onehot['Neighborhood'] = Mine_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Mine_onehot.columns[-1]] + list(Mine_onehot.columns[:-1])
Mine_onehot = Mine_onehot[fixed_columns]

Mine_grouped = Mine_onehot.groupby('Neighborhood').mean().reset_index()
Mine_grouped.head()

,Neighborhood,Zoo,ATM,Accessories Store,Airport,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Apres Ski Bar,...,Warehouse Store,Water Park,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio
0,4J,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,777,0.000000,0.0,0.0,0.2,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,Acton Vale,0.000000,0.0,0.0,0.0,0.0,0.0,0.010101,0.0,0.0,...,0.010101,0.0,0.0,0.0,0.0,0.0,0.010101,0.0,0.0,0.0
3,Allan,0.015152,0.0,0.0,0.0,0.0,0.0,0.015152,0.0,0.0,...,0.015152,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,Amaranth,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [34]:
num_top_venues = 5

for hood in Mine_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Mine_grouped[Mine_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----4J----
                  venue  freq
0  Fast Food Restaurant  0.11
1           Coffee Shop  0.11
2         Grocery Store  0.08
3     Convenience Store  0.08
4                   Pub  0.05


----777----
                  venue  freq
0           Gas Station   0.2
1               Airport   0.2
2  Fast Food Restaurant   0.2
3         Big Box Store   0.2
4  Kitchen Supply Store   0.2


----Acton Vale----
                  venue  freq
0           Coffee Shop  0.07
1  Fast Food Restaurant  0.06
2                 Hotel  0.05
3               Brewery  0.05
4        Sandwich Place  0.05


----Allan----
            venue  freq
0     Coffee Shop  0.08
1  Ice Cream Shop  0.05
2  Breakfast Spot  0.05
3            Café  0.05
4          Bakery  0.05


----Amaranth----
                        venue  freq
0  Construction & Landscaping   0.4
1                       Hotel   0.2
2                    Pharmacy   0.2
3                 Gas Station   0.2
4        Other Great Outdoors   0.0


----Amaruq----
  

                          venue  freq
0  Theme Park Ride / Attraction  0.15
1                         Hotel  0.14
2                          Park  0.04
3                      Mountain  0.04
4                   Coffee Shop  0.04


----Donkin----
                venue  freq
0  Seafood Restaurant  0.14
1                Café  0.14
2              Bakery  0.11
3         Coffee Shop  0.11
4          Restaurant  0.08


----Dundas----
           venue  freq
0           Park  0.10
1  Grocery Store  0.06
2     Restaurant  0.05
3           Café  0.04
4    Coffee Shop  0.04


----Eagle (Dublin Gulch)----
           venue  freq
0    Coffee Shop  0.12
1          Hotel  0.07
2  Grocery Store  0.05
3    Pizza Place  0.05
4       Pharmacy  0.05


----Eagle River----
                  venue  freq
0          Liquor Store  0.67
1        Hunting Supply  0.33
2                   Zoo  0.00
3  Other Great Outdoors  0.00
4  Pakistani Restaurant  0.00


----East Milford----
                venue  freq
0         

                  venue  freq
0           Coffee Shop  0.16
1           Gas Station  0.16
2  Fast Food Restaurant  0.16
3            Restaurant  0.11
4           Supermarket  0.05


----Lalor Lake----
                  venue  freq
0         Moving Target  0.33
1               Airport  0.33
2                  Lake  0.33
3  Other Great Outdoors  0.00
4  Pakistani Restaurant  0.00


----Lamaque----
           venue  freq
0    Gas Station  0.18
1  Grocery Store  0.11
2          Hotel  0.07
3       Pharmacy  0.07
4    Coffee Shop  0.07


----Langlois----
                        venue  freq
0                       Hotel   0.2
1  Construction & Landscaping   0.2
2                        Lake   0.2
3                         Bar   0.2
4                  Print Shop   0.2


----Lanigan----
                  venue  freq
0  Fast Food Restaurant  0.25
1                 Hotel  0.08
2        Sandwich Place  0.08
3        Hardware Store  0.08
4         Grocery Store  0.08


----Laval----
           ven

         venue  freq
0        Hotel   0.1
1         Café   0.1
2  Gas Station   0.1
3          Bar   0.1
4       Resort   0.1


----Renard----
                  venue  freq
0         Grocery Store  0.10
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.07
3              Pharmacy  0.07
4               Brewery  0.05


----Rocanville----
                  venue  freq
0           Gas Station  0.21
1        Ice Cream Shop  0.14
2           Coffee Shop  0.14
3  Fast Food Restaurant  0.14
4                   Inn  0.07


----Saint-Armand----
                 venue  freq
0  American Restaurant  0.05
1                Hotel  0.04
2              Brewery  0.04
3   Italian Restaurant  0.03
4    French Restaurant  0.03


----Saint-Basile----
                  venue  freq
0         Grocery Store  0.07
1                  Park  0.06
2  Fast Food Restaurant  0.06
3                 Hotel  0.05
4                  Café  0.05


----Saint-Modeste----
                  venue  freq
0           Gas Statio

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [53]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Mine_grouped['Neighborhood']

for ind in np.arange(Mine_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Mine_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4J,Fast Food Restaurant,Coffee Shop,Convenience Store,Grocery Store,History Museum,Ski Lodge,Pub,Restaurant,Ski Area,Resort
1,777,Big Box Store,Airport,Fast Food Restaurant,Kitchen Supply Store,Gas Station,Yoga Studio,Duty-free Shop,Eastern European Restaurant,Electronics Store,Escape Room
2,Acton Vale,Coffee Shop,Fast Food Restaurant,Brewery,Hotel,Restaurant,Sandwich Place,Grocery Store,Ice Cream Shop,Hardware Store,Supermarket
3,Allan,Coffee Shop,Breakfast Spot,Restaurant,Bakery,Café,Ice Cream Shop,Pub,Bookstore,Sandwich Place,Grocery Store
4,Amaranth,Construction & Landscaping,Hotel,Gas Station,Pharmacy,Yoga Studio,Exhibit,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store


In [54]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 10

Mine_grouped_clustering = Mine_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Mine_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Mine_merged = df_Mine
Mine_merged.rename(columns={"Mine Name": "Neighborhood"},inplace = True)

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Mine_merged = pd.merge(Mine_merged, neighborhoods_venues_sorted, how="inner", on=["Neighborhood"])
Mine_merged.head() # check the last columns!

,Neighborhood,x,y,Mine Type,Mine Type ID,Latitude,Longitude,marker_color,Town address,Town lat,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New Afton,-1.745331e+06,5.389900e+05,Metal mines,4,50.661000,-120.514000,yellow,"Kamloops , British Columbia",50.675827,...,Hotel,American Restaurant,Gas Station,Coffee Shop,Breakfast Spot,Sushi Restaurant,Café,Brewery,Japanese Restaurant,Restaurant
1,Copper Mountain,-1.804148e+06,4.038439e+05,Metal mines,4,49.331001,-120.533002,yellow,"Fraser Valley , British Columbia",59.715297,...,Train Station,Gift Shop,Coffee Shop,Boat or Ferry,Seafood Restaurant,Historic Site,Border Crossing,Bar,Thai Restaurant,Bookstore
2,Highland Valley,-1.787566e+06,5.359916e+05,Metal mines,4,50.485500,-121.048300,yellow,"Logan Lake , British Columbia",50.494463,...,Gas Station,Coffee Shop,Sushi Restaurant,Café,Hotel,Restaurant,Park,Japanese Restaurant,Brewery,Sandwich Place
3,Gibraltar,-1.773727e+06,7.764453e+05,Metal mines,4,52.528495,-122.287160,yellow,"Cariboo , British Columbia",49.247779,...,Park,Coffee Shop,Hotel,Brewery,Bakery,Ice Cream Shop,Trail,Market,Dessert Shop,Lake
4,Mount Milligan,-1.754343e+06,1.081143e+06,Metal mines,4,55.118000,-124.031000,yellow,"Bulkley-Nechako , British Columbia",54.531617,...,Fast Food Restaurant,Sandwich Place,Warehouse Store,Yoga Studio,Exhibit,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store


In [55]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library


# create map
map_clusters = folium.Map(location=[57, -91], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Mine_merged['Town lat'], Mine_merged['Town long'], Mine_merged['Neighborhood'], Mine_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [57]:
FilePath = R"C:\Users\19029\Desktop\Personal\05 - Learning\Python Learning\IBM\Capstone\MineMergerd.csv"

Mine_merged.to_csv(FilePath, index = False)
